# Multiprocessing

Aby wykorzystać multiprocessing w Pythonie do równoczesnego nasłuchiwania danych z urządzenia szeregowego i przekazywania ich między procesami bez blokowania pierwszego procesu, możemy skorzystać z modułu `multiprocessing.Queue`. Ten moduł umożliwia bezpieczne przekazywanie danych między procesami

In [ ]:
import serial
import multiprocessing

# Funkcja odbierająca dane z portu szeregowego i umieszczająca je w kolejce
def odbierz_dane(port, kolejka):
    while True:
        dane = port.readline().decode().strip()
        if dane:
            kolejka.put(dane)

# Funkcja przetwarzająca dane odebrane z kolejki
def przetwarzaj_dane(kolejka):
    while True:
        if not kolejka.empty():
            dane = kolejka.get()
            # Tutaj możesz wykonywać przetwarzanie na danych
            przetworzone_dane = dane.upper()
            print("Przetworzone dane:", przetworzone_dane)

# Inicjalizacja portu szeregowego
port = serial.Serial('/dev/ttyUSB0', 9600, timeout=1)

# Inicjalizacja kolejki danych
kolejka = multiprocessing.Queue()

# Proces nasłuchujący dane z portu szeregowego
proces_nasluchujacy = multiprocessing.Process(target=odbierz_dane, args=(port, kolejka))
proces_nasluchujacy.daemon = True
proces_nasluchujacy.start()

# Proces przetwarzający dane z kolejki
proces_przetwarzajacy = multiprocessing.Process(target=przetwarzaj_dane, args=(kolejka,))
proces_przetwarzajacy.daemon = True
proces_przetwarzajacy.start()

# Główna pętla programu
while True:
    # Reszta logiki programu
    # ...

    # Tu możesz wykonywać inne operacje, które nie blokują procesów


W powyższym przykładzie, proces odbierz_dane() nasłuchuje danych z portu szeregowego i umieszcza je w kolejce kolejka. Główna pętla programu sprawdza, czy kolejka zawiera dane, a następnie przetwarza je w odpowiedni sposób. Reszta logiki programu może być wykonywana w tej pętli, ponieważ oczekiwanie na dane nie blokuje procesu nasłuchującego.

Przy użyciu multiprocessing.Queue, procesy mogą bezpiecznie komunikować się ze sobą, przekazując dane między kolejnymi procesami. Jest to sposób na efektywne i równoczesne przetwarzanie danych, eliminujący blokowanie procesów.

W tym przykładzie dodano funkcję przetwarzaj_dane(), która jest uruchamiana jako drugi proces. Ta funkcja pobiera dane z kolejki, przetwarza je (w tym przypadku zamienia na wielkie litery) i wyświetla przetworzone dane na konsoli.

Proces nasłuchujący (proces_nasluchujacy) jest odpowiedzialny za odbieranie danych z portu szeregowego i umieszczanie ich w kolejce. Proces przetwarzający (proces_przetwarzajacy) pobiera dane z kolejki i wykonuje na nich operacje przetwarzania.

W głównej pętli programu można dodawać logikę lub inne operacje, które nie blokują działania procesów. Oba procesy (proces_nasluchujacy i proces_przetwarzajacy) działają równocześnie, a komunikacja między nimi odbywa się poprzez współdzieloną kolejkę kolejka.

Czy możesz zmodyfikować powyższy program? Chciałbym, żeby jeden proces nasłuchiwał danych z urządzenia w porcie szeregowym, drugi proces przetwarzał dane i wyświetlał je na bieżąco, a trzeci zapisywał do pliku

In [ ]:
import serial
import multiprocessing

# Funkcja odbierająca dane z portu szeregowego i umieszczająca je w kolejce
def odbierz_dane(port, kolejka):
    while True:
        dane = port.readline().decode().strip()
        if dane:
            kolejka.put(dane)

# Funkcja przetwarzająca dane odebrane z kolejki i wyświetlająca je na bieżąco
def przetwarzaj_dane(kolejka):
    while True:
        if not kolejka.empty():
            dane = kolejka.get()
            # Tutaj możesz wykonywać przetwarzanie na danych
            przetworzone_dane = dane.upper()
            print("Przetworzone dane:", przetworzone_dane)

# Funkcja zapisująca dane do pliku
def zapisz_do_pliku(kolejka):
    with open("dane.txt", "a") as plik:
        while True:
            if not kolejka.empty():
                dane = kolejka.get()
                plik.write(dane + "\n")

# Inicjalizacja portu szeregowego
port = serial.Serial('/dev/ttyUSB0', 9600, timeout=1)

# Inicjalizacja kolejki danych
kolejka = multiprocessing.Queue()

# Proces nasłuchujący dane z portu szeregowego
proces_nasluchujacy = multiprocessing.Process(target=odbierz_dane, args=(port, kolejka))
proces_nasluchujacy.daemon = True
proces_nasluchujacy.start()

# Proces przetwarzający dane z kolejki i wyświetlający je na bieżąco
proces_przetwarzajacy = multiprocessing.Process(target=przetwarzaj_dane, args=(kolejka,))
proces_przetwarzajacy.daemon = True
proces_przetwarzajacy.start()

# Proces zapisujący dane do pliku
proces_zapisujacy = multiprocessing.Process(target=zapisz_do_pliku, args=(kolejka,))
proces_zapisujacy.daemon = True
proces_zapisujacy.start()

# Główna pętla programu
while True:
    # Reszta logiki programu
    # ...

    # Tu możesz wykonywać inne operacje, które nie blokują procesów


W tym przykładzie dodano trzeci proces proces_zapisujacy, który jest odpowiedzialny za zapisywanie danych z kolejki do pliku "dane.txt". Każdy odczytany wiersz jest zapisywany jako osobna linia w pliku.

Proces nasłuchujący (proces_nasluchujacy) odbiera dane z portu szeregowego i umieszcza je w kolejce. Proces przetwarzający (proces_przetwarzajacy) przetwarza dane z kolejki i wyświetla je na bieżąco. Proces zapisujący (proces_zapisujacy) zapisuje dane z kolejki do pliku.

W głównej pętli programu można dodawać logikę lub inne operacje, które nie blokują działania procesów. Wszystkie